In [1]:
from pandas._libs.tslibs.timestamps import Timestamp
end_tbv=Timestamp('2022-11-05 00:00:00')
# end_tbv=Timestamp('2022-11-05 00:00:00', freq='MS')
end_tbv

Timestamp('2022-11-05 00:00:00')

In [2]:
import string,os
string.punctuation
# from assets.mymodule.my_db_cred import host, password, user
import mysql.connector as sql_db
import pandas as pd
import warnings as w
w.filterwarnings('ignore')

In [3]:
query1 = '''
SELECT 
     date(a.CreatedDate) As Date,
    fa.FacilityName as Location,   
    count(distinct date(a.CreatedDate)) As Camps_Conducted,
    COUNT(DISTINCT a.VisitCode) AS 'Total Beneficiaries Registered',
    COUNT(DISTINCT case when a.VisitNo=1 then a.VisitCode end) AS 'Total Unique Registered',
    COUNT(DISTINCT case when a.VisitNo>1 then a.VisitCode end) AS 'Total Revisits',
       COUNT(DISTINCT case when (d.ProcedureID in (29,30,31,32) or ph.rbs is not null or ph.spo2 is not null 
        or ph.DiastolicBP_1stReading is not null or ph.SystolicBP_1stReading is not null) then a.VisitCode end) AS 'Total Beneficiaries taken Lab tests',
    COUNT(DISTINCT c.VisitCode) AS '#ben to Drug Prescribed',
    COUNT(distinct case when (nc.NCD_Condition='Hypertension' or p.DiagnosisProvided like 'Hypertension' or
    ((ph.DiastolicBP_1stReading>90 or ph.SystolicBP_1stReading>140) and ph.rbs<200) ) then a.VisitCode end) As 'HYpertension',
    COUNT(distinct case when nc.NCD_Condition='Diabetes Mellitus' or p.DiagnosisProvided like'Diabetes Mellitus' or ((ph.DiastolicBP_1stReading<=90 or ph.SystolicBP_1stReading<=140) and ph.rbs>200)
    or (d.ProcedureID in (29) and d.TestResultValue>200) then a.VisitCode end) As 'Diabetes',
    COUNT(DISTINCT case when (nc.NCD_Condition='Diabetes Mellitus' AND nc.NCD_Condition='Hypertension') or ((ph.DiastolicBP_1stReading>90 or ph.SystolicBP_1stReading>140) and ph.rbs>200) 
     or (p.DiagnosisProvided like'%Diabetes%' and p.DiagnosisProvided like '%Hypertension%')
      then a.VisitCode end) As 'HTN & DM',  
	count(distinct case when p.DiagnosisProvided like '%Cancer%' then a.visitcode end) As Cancer,
       COUNT(DISTINCT CASE
            WHEN d.ProcedureID=32 THEN d.VisitCode
        END) AS 'Urine Sugar',
    COUNT(DISTINCT CASE
            WHEN d.ProcedureID=31 THEN d.VisitCode
        END) AS 'Urine Albumin',
        COUNT(DISTINCT CASE
            WHEN (d.ProcedureID=29 or ph.rbs is not null) THEN a.VisitCode
        END) AS 'RBS',
    COUNT(DISTINCT CASE
            WHEN d.ProcedureID =30 THEN d.VisitCode
        END) AS 'HbA1c',
        COUNT(DISTINCT CASE
            WHEN (ph.SystolicBP_1stReading is not null or ph.DiastolicBP_1stReading is not null) THEN a.VisitCode
        END) AS 'BP'
        
FROM
    db_iemr.t_benvisitdetail a
        INNER JOIN
db_iemr.i_ben_flow_outreach e ON a.VisitCode = e.beneficiary_visit_code
        LEFT JOIN
    db_iemr.t_benclinicalobservation b ON a.VisitCode = b.VisitCode
        LEFT JOIN
    db_iemr.t_prescription p on a.VisitCode=p.VisitCode    
        left join 
    db_iemr.t_ncddiagnosis nc on a.VisitCode=nc.VisitCode    
    left join db_iemr.t_phy_vitals ph on ph.VisitCode=a.VisitCode
        LEFT JOIN
        db_iemr.t_cancerdiagnosis ca on ca.visitcode=a.visitcode
        left join
    db_iemr.t_prescribeddrug c ON a.VisitCode = c.VisitCode
        LEFT JOIN
    db_iemr.t_lab_testresult d ON a.VisitCode = d.VisitCode
        LEFT JOIN
    db_iemr.t_patientissue f ON a.VisitCode = f.VisitCode
           left join db_iemr.m_van v on a.VanID=v.VanID
    left join db_iemr.m_facility fa on v.FacilityID=fa.FacilityID
WHERE
    a.CreatedDate>='2022-02-01 00:00:00' and a.ProviderServiceMapID=3
GROUP BY date(a.CreatedDate),fa.FacilityName;
'''

In [4]:
#Class For Fetching data From Query
class Sanofi:
    def __init__(self,query):
        self.query=query
        
    #Database connections
    def mycon(self,db_name:str):
        try:
            self.mydb = sql_db.connect(host="192.168.45.221",user="Rajesh_s",password="Rajesh_s@2022$", database=db_name,auth_plugin="mysql_native_password")
            return self.mydb
        except Exception as e:
            print(e)
            print("An error has been occured In Exception Block..!")
    
    def data_frame(self):
        self.db_name="db_iemr"
        self.mydb = self.mycon(self.db_name)

        self.mycursor = self.mydb.cursor()
        self.mycursor.execute(self.query)

        self.df=pd.DataFrame(self.mycursor, columns=[i[0] for i in self.mycursor.description])
        self.df.drop_duplicates(inplace=True)
        self.df['Date'] = pd.to_datetime(self.df['Date'], format='%Y-%m-%d')
        self.df.dropna(inplace=True)
        return self.df
                
#Object  Creation
obj=Sanofi(query1)

In [5]:
df_main=obj.data_frame()
df_main

,Date,Location,Camps_Conducted,Total Beneficiaries Registered,Total Unique Registered,Total Revisits,Total Beneficiaries taken Lab tests,#ben to Drug Prescribed,HYpertension,Diabetes,HTN & DM,Cancer,Urine Sugar,Urine Albumin,RBS,HbA1c,BP
0,2022-02-03,SANOFI JALAGAON MMU 2,1,2,2,0,0,1,0,0,0,0,0,0,0,0,0
1,2022-02-03,SANOFI RATNAGIRI MMU 1,1,1,1,0,1,1,0,0,0,0,1,1,1,1,0
2,2022-02-03,SANOFI SINDHUDURG MMU 1,1,1,1,0,1,0,0,0,0,0,1,1,1,1,1
3,2022-02-04,SANOFI JALAGAON MMU 1,1,24,24,0,24,0,1,4,0,0,0,0,17,0,22
4,2022-02-04,SANOFI JALAGAON MMU 2,1,2,2,0,1,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5073,2023-01-11,SANOFI SINDHUDURG MMU 1,1,43,7,36,43,32,23,10,4,0,0,0,43,0,43
5074,2023-01-11,SANOFI SINDHUDURG MMU 2,1,16,3,13,16,12,5,2,1,0,1,1,16,1,16
5075,2023-01-11,SANOFI SINDHUDURG MMU 3,1,2,1,1,2,2,2,0,0,0,0,0,2,0,2
5076,2023-01-11,SANOFI THANE MMU 1,1,42,41,1,42,5,5,3,1,0,0,0,42,1,42


In [6]:
df_main.columns

Index(['Date', 'Location', 'Camps_Conducted', 'Total Beneficiaries Registered',
       'Total Unique Registered', 'Total Revisits',
       'Total Beneficiaries taken Lab tests', '#ben to Drug Prescribed',
       'HYpertension', 'Diabetes', 'HTN & DM', 'Cancer', 'Urine Sugar',
       'Urine Albumin', 'RBS', 'HbA1c', 'BP'],
      dtype='object')

In [7]:
df_main['Total Beneficiaries Registered'].sum()

210689

In [8]:
df_main['Total Unique Registered'].sum()

149743